# Projeto Covid - Situação Brasil
## Por: Michel Souza Santana
> Data: 23/01/2023

### Objetivos
* Requisitar de um sites on-line via url dados sobre a covid19, vacinação, população, estados mais afetados, casos x mortes, etc.
* Armazenar os dados capturados em uma tabela.
* Organizar e limpar os dados.
* Anlisar os dados e selecionar os mais relevantes para o projeto.

### Análise dos dados
* Letalidade
* Taxa de mortalidade
* Índice de vacinação
* Casos X Mortes X Vacinação
* estados mais afetados
* População total

## Questão de Negócio
> Com base na ánálise verificar a contribuição das vacinas no resultado geeral de casos x mortes.

## Entendimento do Negócio
> ...

## Carregamento das bibliotecas Python

In [176]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt

## Coleta de Dados & Limpeza dos Dados
> (Casos x Morte): https://news.google.com/covid19/map?hl=pt-BR&mid=%2Fm%2F015fr&gl=BR&ceid=BR%3Apt-419 (Dados atualizados em: 21/01/2023)

> (Vacinação): https://especiais.g1.globo.com/bemestar/vacina/2021/mapa-brasil-vacina-covid/ (Dados atualizado em: 23/01/2023)

### Coletando dados de casos x mortes

In [177]:
html = urlopen('https://news.google.com/covid19/map?hl=pt-BR&mid=%2Fm%2F015fr&gl=BR&ceid=BR%3Apt-419')
bs = BeautifulSoup(html.read(), 'html.parser')
dados = bs.findAll('tr', {'class': 'sgXwHf wdLSAe YvL7re'})
dados[:2]

[<tr class="sgXwHf wdLSAe YvL7re" data-id="/m/01hd58" jsaction="click:KoQL8" jslog="110764; track:click"><th class="l3HOY" role="rowheader" scope="row"><div class="TWa0lb"><div class="ggJ3df" jsaction="JIbuQc:ROTNpb" jslog="113725; track:click"><button aria-label="Liberada" class="VfPpkd-Bz112c-LgbsSe yHy1rc eT1oJ mN1ivc Vs1w5b" data-disable-idom="true" data-n-mid="/m/01hd58" jsaction="click:cOuCgd; mousedown:UX7yZ; mouseup:lbsD7e; mouseenter:tfO1Yc; mouseleave:JywGue; touchstart:p6p2H; touchmove:FwuNnf; touchend:yfqBxc; touchcancel:JMtRjd; focus:AHmuwe; blur:O22p3e; contextmenu:mg9Pef;mlnRJb:fLiPzd;" jscontroller="soHxf"><div class="VfPpkd-Bz112c-Jh9lGc" jsname="s3Eaab"></div><div class="VfPpkd-Bz112c-J1Ukfc-LhBDec"></div><span aria-hidden="true" class="VfPpkd-kBDsod gsIK3e"><svg class="Xa3RBb NMm5M" focusable="false" height="16" viewbox="0 0 24 24" width="16"><path d="M0 0h24v24H0z" fill="none" stroke="#000" stroke-opacity=".008" stroke-width="0"></path><path d="M17 4v7l2 3v2h-6v5l-1

> Criando DataFrame (Casos x Mortes)

In [178]:
new_data = []
for d in dados:
    data = []
    td = []
    data.append(d.find('div', {'class': 'pcAJd'}).get_text())
    td = d.findAll('td', {'class': 'l3HOY'})
    data.append(td[0].get_text())
    data.append(td[1].get_text())
    data.append(td[3].get_text())
    data.append(td[4].get_text())
    new_data.append(data)
new_data[:5]

[['São Paulo', '6.389.460', '0', '145.099', '178.178'],
 ['Minas Gerais', '4.122.029', '0', '197.518', '64.799'],
 ['Rio Grande do Sul', '2.940.791', '0', '260.558', '41.716'],
 ['Paraná', '2.904.979', '179', '262.142', '45.929'],
 ['Rio de Janeiro', '2.719.059', '0', '165.180', '76.636']]

In [179]:
df_casos = pd.DataFrame(new_data)
df_casos.head()

,0,1,2,3,4
0,São Paulo,6.389.460,0,145.099,178.178
1,Minas Gerais,4.122.029,0,197.518,64.799
2,Rio Grande do Sul,2.940.791,0,260.558,41.716
3,Paraná,2.904.979,179,262.142,45.929
4,Rio de Janeiro,2.719.059,0,165.180,76.636


In [180]:
df_casos = pd.DataFrame(new_data, columns=['local', 'total_casos', 'novos_por_dia', 'a_cada_milhao', 'total_mortes' ])
df_casos.head()

,local,total_casos,novos_por_dia,a_cada_milhao,total_mortes
0,São Paulo,6.389.460,0,145.099,178.178
1,Minas Gerais,4.122.029,0,197.518,64.799
2,Rio Grande do Sul,2.940.791,0,260.558,41.716
3,Paraná,2.904.979,179,262.142,45.929
4,Rio de Janeiro,2.719.059,0,165.180,76.636


In [181]:
df_casos.to_csv('csv/casosXmortes.csv')

### Coletando dados das vascinas

> Devido à dificuldade de interação sobre os dados via url, copiei manualmente o html da página e extrair os dados necessários.

In [182]:
html = pd.read_html('html_vacinacao.html')
df_vacinas = pd.DataFrame(html[0])
df_vacinas.head()

,Estado,2ª doseou doseúnica(Unidades),2ª doseou doseúnica(Em %),Dose dereforço*(Unidades),Dose dereforço*(Em %),1ª dose(Unidade),1ª dose(Em %),Total devacinasaplicadas
0,Acre,597.935,"65,05%",284.813,"30,99%",696.619,"75,79%",1.280.931
1,Alagoas,2.355.718,"69,73%",1.272.731,"37,67%",2.642.076,"78,21%",4.941.707
2,Amazonas,2.922.899,"67,48%",1.691.944,"39,06%",3.361.647,"77,61%",6.209.127
3,Amapá,514.824,"57,63%",269.240,"30,14%",614.022,"68,74%",1.111.564
4,Bahia,11.226.993,"74,66%",7.710.188,"51,28%",11.717.323,"77,93%",22.679.938


In [183]:
df_vacinas['2ª doseou doseúnica(Unidades)'] = df_vacinas['2ª doseou doseúnica(Unidades)'].str.replace('.', '')
df_vacinas['2ª doseou doseúnica(Unidades)'] = df_vacinas['2ª doseou doseúnica(Unidades)'].astype('int64')
df_vacinas['2ª doseou doseúnica(Em %)'] = df_vacinas['2ª doseou doseúnica(Em %)'].str.replace(',', '.')
df_vacinas['2ª doseou doseúnica(Em %)'] = df_vacinas['2ª doseou doseúnica(Em %)'].str.replace('%', '')
df_vacinas['2ª doseou doseúnica(Em %)'] = df_vacinas['2ª doseou doseúnica(Em %)'].astype(float)
df_vacinas['Dose dereforço*(Unidades)'] = df_vacinas['Dose dereforço*(Unidades)'].str.replace('.', '')
df_vacinas['Dose dereforço*(Unidades)'] = df_vacinas['Dose dereforço*(Unidades)'].astype('int64')
df_vacinas['Dose dereforço*(Em %)'] = df_vacinas['Dose dereforço*(Em %)'].str.replace(',', '.')
df_vacinas['Dose dereforço*(Em %)'] = df_vacinas['Dose dereforço*(Em %)'].str.replace('%', '')
df_vacinas['Dose dereforço*(Em %)'] = df_vacinas['Dose dereforço*(Em %)'].astype(float)
df_vacinas['1ª dose(Unidade)'] = df_vacinas['1ª dose(Unidade)'].str.replace('.', '')
df_vacinas['1ª dose(Unidade)'] = df_vacinas['1ª dose(Unidade)'].astype('int64')
df_vacinas['1ª dose(Em %)'] = df_vacinas['1ª dose(Em %)'].str.replace(',', '.')
df_vacinas['1ª dose(Em %)'] = df_vacinas['1ª dose(Em %)'].str.replace('%', '')
df_vacinas['1ª dose(Em %)'] = df_vacinas['1ª dose(Em %)'].astype(float)
df_vacinas['Total devacinasaplicadas'] = df_vacinas['Total devacinasaplicadas'].str.replace('.', '')
df_vacinas['Total devacinasaplicadas'] = df_vacinas['Total devacinasaplicadas'].astype('int64')

/tmp/ipykernel_1028820/3013469470.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_vacinas['2ª doseou doseúnica(Unidades)'] = df_vacinas['2ª doseou doseúnica(Unidades)'].str.replace('.', '')
/tmp/ipykernel_1028820/3013469470.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_vacinas['Dose dereforço*(Unidades)'] = df_vacinas['Dose dereforço*(Unidades)'].str.replace('.', '')
/tmp/ipykernel_1028820/3013469470.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_vacinas['1ª dose(Unidade)'] = df_vacinas['1ª dose(

In [184]:
df_vacinas = df_vacinas.rename(columns={'Estado': 'estado', 
                   '2ª doseou doseúnica(Unidades)': '2ª dose', 
                   '2ª doseou doseúnica(Unidades)': '2ª dose',
                   '2ª doseou doseúnica(Em %)': '2ª dose %',
                   'Dose dereforço*(Unidades)': 'reforco',
                   'Dose dereforço*(Em %)': 'reforco %',
                   '1ª dose(Unidade)': '1ª dose',
                   '1ª dose(Em %)': '1ª dose %',
                   'Total devacinasaplicadas': 'total vascinas'
                   }) 

In [185]:
df_vacinas.head()

,estado,2ª dose,2ª dose %,reforco,reforco %,1ª dose,1ª dose %,total vascinas
0,Acre,597935,65.05,284813,30.99,696619,75.79,1280931
1,Alagoas,2355718,69.73,1272731,37.67,2642076,78.21,4941707
2,Amazonas,2922899,67.48,1691944,39.06,3361647,77.61,6209127
3,Amapá,514824,57.63,269240,30.14,614022,68.74,1111564
4,Bahia,11226993,74.66,7710188,51.28,11717323,77.93,22679938


In [186]:
df_vacinas.to_csv('csv/vascinas.csv')

### Coletando dados da população

In [187]:
html = urlopen('https://www.amunes.org.br/noticia/ler/173/ibge-divulga-nova-estimativa-populacional-dos-estados-e-municipios')
bs = BeautifulSoup(html.read(), 'html.parser')
populacao = bs.findAll('tr')

In [188]:
tabela = []
for t in range(len(populacao)):
    tabela.append(populacao[t])

In [189]:
df_populacao = pd.DataFrame(tabela)
df_populacao.drop(index=([0, 1]), inplace=True)
df_populacao = df_populacao.reset_index()
df_populacao.drop(columns=['index'], inplace=True)

/usr/lib/python3/dist-packages/pandas/core/internals/construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [190]:
for i in range(len(df_populacao[0])):
    df_populacao[0][i] = df_populacao[0][i].get_text()
    df_populacao[1][i] = df_populacao[1][i].get_text()
df_populacao.head()

,0,1
0,São Paulo,43.663.672
1,Minas Gerais,20.593.366
2,Rio de Janeiro,16.369.178
3,Espírito Santo,3.839.363
4,Região Nordeste,


In [191]:
df_populacao[1] = df_populacao[1].str.replace('.', '')
df_populacao = df_populacao.fillna(0)
df_populacao.head()

/tmp/ipykernel_1028820/1068304898.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_populacao[1] = df_populacao[1].str.replace('.', '')


,0,1
0,São Paulo,43663672
1,Minas Gerais,20593366
2,Rio de Janeiro,16369178
3,Espírito Santo,3839363
4,Região Nordeste,


In [192]:
for i in range(len(df_populacao[0])):
    if df_populacao[1][i] == '' or df_populacao[1][i] == ' ':
        df_populacao[1][i] = 0

In [193]:
df_populacao[1] = df_populacao[1].astype('int64')
df_populacao.head()

,0,1
0,São Paulo,43663672
1,Minas Gerais,20593366
2,Rio de Janeiro,16369178
3,Espírito Santo,3839363
4,Região Nordeste,0


In [194]:
df_populacao = df_populacao[df_populacao[1] != 0]
df_populacao.head()

,0,1
0,São Paulo,43663672
1,Minas Gerais,20593366
2,Rio de Janeiro,16369178
3,Espírito Santo,3839363
5,Bahia,15044127


In [195]:
df_populacao = df_populacao.reset_index()
df_populacao.drop(columns=['index'], inplace=True)
df_populacao = df_populacao.rename(columns={df_populacao.columns[0]:'estado'})
df_populacao = df_populacao.rename(columns={df_populacao.columns[1]:'populacao'})

df_populacao = df_populacao.sort_values(by=['estado'])
df_populacao = df_populacao.reset_index()
df_populacao.drop(columns=['index'], inplace=True)
df_populacao.head()

,estado,populacao
0,Acre,776463
1,Alagoas,3300938
2,Amapá,734995
3,Amazonas,3807923
4,Bahia,15044127


In [196]:
df_populacao.to_csv('csv/populacao.csv')

## Exploração dos Dados
> ...

## Modelagem dos Dados
> ...

## Algoritmos de Machine Laearning
> ...

## Avaliação do algoritimo
> ...

## Storytelling
> ...